##Spark DataFrames

In [ ]:
spark

In [ ]:
from pyspark.sql.types import *  # Necessary for creating schemas
from pyspark.sql.functions import * # Importing PySpark functions

###The 'groupBy' Function and Aggregations

The `groupBy()` function groups the DataFrame using the specified columns, then, we can run aggregation on them. The available aggregate functions are:

- `count()`: counts the number of records for each group
- `sum()`: compute the sum for each numeric column for each group
- `min()`: computes the minimum value for each numeric column for each group
- `max()`: computes the maximum value for each numeric column for each group
- `avg()` or `mean()`: computes average values for each numeric columns for each group
- `pivot()`: pivots a column of the current DataFrame and perform the specified aggregation

Before we get into aggregations, let's load in a **CSV** with interesting data and create a new DataFrame.

You do this with the `spark-csv` package. Documentation on that is available at:
- https://github.com/databricks/spark-csv

The dataset that will be loaded in to demonstrate contains data about flights departing New York City airports (`JFK`, `LGA`, `EWR`) in 2013. It has 336,776 rows and 16 columns.

There are more time-based functions:
- `date_sub()`: subtract an integer number of days from a *Date* or *Timestamp*
- `date_add()`: add an integer number of days from a *Date* or *Timestamp*
- `datediff()`: get the difference between two dates
- `add_months()`: add an integer number of months
- `months_between()`: get the number of months between two dates
- `next_day()`: returns the first date which is later than the value of the date column
- `last_day()`: returns the last day of the month which the given date belongs to
- `dayofmonth()`: extract the day of the month of a given date as integer
- `dayofyear()`: extract the day of the year of a given date as integer
- `weekofyear()`: extract the week number of a given date as integer
- `quarter()`: extract the quarter of a given date

###Joins

Joins are easily performed with Spark DataFrames. The expression is:

`join(other, on = None, how = None)`

where:
- other: a DataFrame that serves as the right side of the join
- on: typically a join expression
- how: the default is `inner` but there are also `inner`, `outer`, `left_outer`, `right_outer`, and `leftsemi` joins available

Let's load in some more data so that we can have two DataFrames to join. The **CSV** file `weather.csv` contains hourly meteorological data from EWR during 2013.

In [51]:
# Create a schema object...
weather_schema = StructType([  
  StructField('year', IntegerType(), True),
  StructField('month', IntegerType(), True),
  StructField('day', IntegerType(), True),
  StructField('hour', IntegerType(), True),
  StructField('temp', FloatType(), True),
  StructField('dewp', FloatType(), True),
  StructField('humid', FloatType(), True),
  StructField('wind_dir', IntegerType(), True),
  StructField('wind_speed', FloatType(), True),
  StructField('wind_gust', FloatType(), True),
  StructField('precip', FloatType(), True),
  StructField('pressure', FloatType(), True),
  StructField('visib', FloatType(), True)
  ])

#...and then read the CSV with the schema
weather = \
(sqlContext
 .read
 .format('com.databricks.spark.csv')
 .schema(weather_schema)
 .options(header = True)
 .load('s3://bigdatatoolscloud/weather.csv'))

In [52]:
# Have a look at the imported dataset
display(weather)

In [53]:
# We need those `month`, `day`, and `hour` values back
nycflights = \
(nycflights
 .withColumn('month', month(nycflights.timestamp))
 .withColumn('day', dayofmonth(nycflights.timestamp))
 .withColumn('hour', hour(nycflights.timestamp)))

# Join the `nycflights` DF with the `weather` DF 
nycflights_all_columns = \
(nycflights
 .join(weather,
       [nycflights.month == weather.month, # three join conditions: month,
        nycflights.day == weather.day,     #                        day,
        nycflights.hour == weather.hour],  #                        hour
       'left_outer')) # left outer join: keep all rows from the left DF (flights), with the matching rows in the right DF (weather)
                      # NULLs created if there is no match to the right DF

In [54]:
# Notice that lots of columns created, as well as duplicate column names (not a bug! a feature?)
display(nycflights_all_columns)

In [55]:
# One way to reduce the number of extraneous
# columns is to use a `select()` statement
nycflights_wind_visib = \
(nycflights_all_columns
 .select(['timestamp', 'carrier', 'flight',
          'origin', 'dest', 'wind_dir',
          'wind_speed', 'wind_gust', 'visib']))

In [56]:
# Examine the DataFrame, now with less columns
display(nycflights_wind_visib)

Let's load in even more data so we can determine if any takeoffs occurred in very windy weather.

The **CSV** `beaufort_land.csv` contains Beaufort scale values (the `force` column), wind speed ranges in *mph*, and the name for each wind force.

In [58]:
# Create a schema object... 
beaufort_land_schema = StructType([  
  StructField('force', IntegerType(), True),
  StructField('speed_mi_h_lb', IntegerType(), True),
  StructField('speed_mi_h_ub', IntegerType(), True),
  StructField('name', StringType(), True)
  ])

# ...and then read the CSV with the schema
beaufort_land = \
(sqlContext
 .read
 .format('com.databricks.spark.csv')
 .schema(beaufort_land_schema)
 .options(header = True)
 .load('s3://bigdatatoolscloud/beaufort_land.csv'))
 

In [59]:
# Have a look at the imported dataset
display(beaufort_land)

In [60]:
# Join the current working DF with the `beaufort_land` DF
# and use join expressions that use the WS ranges
nycflights_wind_visib_beaufort = \
(nycflights_wind_visib
 .join(beaufort_land,
      [nycflights_wind_visib.wind_speed >= beaufort_land.speed_mi_h_lb,
       nycflights_wind_visib.wind_speed < beaufort_land.speed_mi_h_ub],
       'left_outer')
 .withColumn('month', month(nycflights_wind_visib.timestamp)) # Create a month column from `timestamp` values
 .drop('speed_mi_h_lb')
 .drop('speed_mi_h_ub')
)

In [61]:
# View the joined DF; now we have extra data on wind speed!
display(nycflights_wind_visib_beaufort)

In [62]:
# We can inspect the number of potentially dangerous
# takeoffs (i.e., where the Beaufort force is high)
# month-by-month through the use of the `crosstab()` function
crosstab_month_force = \
(nycflights_wind_visib_beaufort
 .crosstab('month', 'force'))

# After creating the crosstab DataFrame, use a few
# functions to clean up the resultant DataFrame
crosstab_month_force = \
(crosstab_month_force
 .withColumn('month_force',
             crosstab_month_force.month_force.cast('int')) # the column is initially a string but recasting as
                                                           # an `int` will aid ordering in the next expression
 .orderBy('month_force')
 .drop('null'))

In [63]:
# Display the cross tabulation; turns out January was a bit riskier for takeoffs due to wind conditions
display(crosstab_month_force)

###User Defined Functions (UDFs)

**UDF**s allow for computations of values while looking at every input row in the DataFrame. They allow you to make your own function and import functionality from other **Python** libraries.

In [65]:
# Define a function to convert velocity from
# miles per hour (mph) to meters per second (mps)
def mph_to_mps(mph):
  mps = mph * 0.44704
  return mps

# Register this function as a UDF using `udf()`
mph_to_mps = udf(mph_to_mps, FloatType()) # An output type was specified

In [66]:
# Create two new columns that are conversions of wind
# speeds from mph to mps
display(
  nycflights_wind_visib_beaufort
  .withColumn('wind_speed_mps', mph_to_mps('wind_speed'))
  .withColumn('wind_gust_mps', mph_to_mps('wind_gust'))
  .withColumnRenamed('wind_speed', 'wind_speed_mph')
  .withColumnRenamed('wind_gust', 'wind_gust_mph')
)

###Writing DataFrames to Files
We can easily write DataFrame data to a variety of different file formats.

In [68]:
# Saving to CSV is quite similar to reading from a CSV file
(crosstab_month_force
 .write
 .mode('overwrite')
 .format('com.databricks.spark.csv')
 .save('s3://bigdatatoolscloud/'))

In [69]:
# Saving to Parquet is generally recommended for later retrieval
(crosstab_month_force
 .write
 .mode('overwrite')
 .parquet('s3://bigdatatoolscloud/'))

###Useful Links

There are many more functions... although I tried to cover a lot of ground, there are dozens more functions for DataFrames that I haven't touched upon.

The main project page for Spark:

- http://spark.apache.org

The main reference for PySpark is:

- http://spark.apache.org/docs/latest/api/python/index.html

These examples are available at:

- https://github.com/rich-iannone/so-many-pyspark-examples

Information on the Parquet file format can be found at its project page:

- http://parquet.apache.org

The GitHub project page for `spark-csv` package; contains usage documentation:

- https://github.com/databricks/spark-csv